Compiling cython code

In [2]:
!python setup.py build_ext --inplace

Compiling preprocessing.pyx because it changed.
[1/1] Cythonizing preprocessing.pyx
running build_ext
building 'preprocessing' extension
creating build
creating build/temp.linux-x86_64-2.7
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fno-strict-aliasing -Wdate-time -D_FORTIFY_SOURCE=2 -g -fstack-protector-strong -Wformat -Werror=format-security -fPIC -I/usr/local/lib/python2.7/dist-packages/numpy/core/include -I. -I/usr/include/python2.7 -c preprocessing.c -o build/temp.linux-x86_64-2.7/preprocessing.o
In file included from /usr/local/lib/python2.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1809:0,
                 from /usr/local/lib/python2.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /usr/local/lib/python2.7/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from preprocessing.c:444:
/usr/local/lib/python2.7/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: 

In [ ]:
import time
import numpy as np
import mxnet as mx
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
from preprocessing import data_iterator_cython
import logging
import sys, random, time, math
from collections import namedtuple
from operator import itemgetter

In [3]:
corpus = Text8Corpus("text8")
current_time = time.time()
model = Word2Vec(iter=1, sg=1)
model.build_vocab(corpus)
print "Building vocab took %s seconds" % (time.time() - current_time)

Building vocab took 7.36954212189 seconds


In [4]:
batch_data = []
batch_label = []
batch_label_weight = []

In [6]:
current_time = time.time()
job_batch, batch_size = [], 0
for sent_idx, sentence in enumerate(corpus):
    sentence_length = model._raw_word_count([sentence])

    # can we fit this sentence into the existing job batch?
    if batch_size + sentence_length <= model.batch_words:
        # yes => add it to the current job
        job_batch.append(sentence)
        batch_size += sentence_length
    else:
        sents = data_iterator_cython(model, job_batch, model.alpha)
        for sent in sents:
            batch_data.append(sent[0])
            batch_label.append(sent[1:])
        job_batch[:] = []
        batch_size = 0
print time.time() - current_time
print "data prep took: ", time.time() - current_time

61.8788969517
data prep took:  61.8791129589


In [7]:
batch_data = mx.nd.array(batch_data)
batch_label = mx.nd.array(batch_label)

In [8]:
target_weight = mx.nd.zeros((batch_data.shape[0], model.negative+1))
target_weight[:,0] = 1

In [9]:
batch_data = batch_data.reshape((batch_data.shape[0],1))

In [63]:
batch_size = 256

In [64]:
nd_iter = mx.io.NDArrayIter(data = {"center_word" : batch_data, "target_words": batch_label},
                            label={ "labels":target_weight},
                            batch_size=batch_size, shuffle = True)

In [65]:
neg_dim = model.negative
vocab = len(model.wv.vocab)
dim = model.vector_size

In [66]:
def get_sym_makeloss(vocab_size, dim, batch_size):
    labels = mx.sym.Variable('labels') #1 positive and k "0" labels
    center_word = mx.sym.Variable('center_word')
    target_words = mx.sym.Variable('target_words') # 1 target + k negative samples
    center_vector = mx.sym.Embedding(data = center_word, input_dim = vocab_size,
                                  output_dim = dim, name = 'syn0_embedding')
    target_vectors = mx.sym.Embedding(data = target_words, input_dim = vocab_size,
                                   output_dim = dim, name = 'syn1_embedding')
    pred = mx.sym.broadcast_mul(center_vector, target_vectors)
    pred = mx.sym.sum(data = pred, axis = 2)
    sigmoid = mx.sym.sigmoid(pred)
    loss = mx.sym.sum(labels * mx.sym.log(sigmoid) + (1 - labels) * mx.sym.log(1 - sigmoid), axis=1)
    loss *= -1.0 / batch_size 
    loss_layer = mx.sym.MakeLoss(loss)
    return loss_layer

In [68]:
sym = get_sym_makeloss(vocab, dim, batch_size)

In [ ]:
network = mx.mod.Module(sym, data_names=("center_word", "target_words",), label_names=("labels",),context=mx.gpu())
network.bind(data_shapes=nd_iter.provide_data, label_shapes=nd_iter.provide_label)
current_time = time.time()
network.fit(nd_iter, num_epoch=1,optimizer=mx.optimizer.Adam(learning_rate=0.025),
            eval_metric=mx.metric.Torch(),initializer=mx.initializer.Uniform(scale=.01))
print time.time() - current_time

In [70]:
all_vecs = network.get_params()[0]["syn0_embedding_weight"].asnumpy()

In [25]:
model.wv.syn0 = all_vecs
model.wv.syn0norm = all_vecs

In [27]:
model.most_similar("sun")

[(u'wind', 1.0105350017547607),
 (u'calendar', 0.9821910858154297),
 (u'faster', 0.9229880571365356),
 (u'leap', 0.9178435802459717),
 (u'output', 0.9089897871017456),
 (u'seconds', 0.9083570837974548),
 (u'moon', 0.9029580354690552),
 (u'orbit', 0.8970711827278137),
 (u'week', 0.8915328979492188),
 (u'reach', 0.8870389461517334)]